In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import Model
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dropout, Dense, Conv2D, MaxPooling2D, LeakyReLU , GlobalMaxPooling2D
from tensorflow.keras.models import Sequential, load_model
import matplotlib.pyplot as plt
import os

C:\Users\Janmejai\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Janmejai\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Janmejai\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
train_dir = os.path.join('data', 'train')
val_dir = os.path.join('data', 'val')


In [3]:
mobile = MobileNetV2(weights = "imagenet", include_top = False, input_shape = (224,224,3))

for layer in mobile._layers:
    layer.trainable = False
x = mobile.output
x = Flatten(name = "flatten")(x)
x = Dropout(0.4)(x)
x = Dense(2, activation = 'softmax')(x)

model = Model(inputs=mobile.input, outputs = x)

# If you want to retrain : 
# from tensorflow.keras.models import load_model
# model = load_model('model_name.h5')

In [4]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 20,
                                   height_shift_range=0.3,
                                   width_shift_range= 0.3,
                                   horizontal_flip = True,
                                   zoom_range = 0.3,
                                   shear_range = 0.1,
                                   fill_mode = 'nearest'
                                )

valid_datagen = ImageDataGenerator(rescale = 1./255)

train_images = train_datagen.flow_from_directory(train_dir,
                                                target_size = (224,224),
                                                batch_size = 25,
                                                class_mode = 'categorical',
                                                )
val_images = valid_datagen.flow_from_directory(val_dir,
                                               target_size = (224,224),
                                               batch_size = 15,
                                               class_mode = 'categorical',
                                              )


Found 712 images belonging to 2 classes.
Found 179 images belonging to 2 classes.


In [6]:
# model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics= ['accuracy'])
import tensorflow
initial_lr = 1e-3
loss = "categorical_crossentropy"
model.compile(tensorflow.keras.optimizers.Adam(lr=initial_lr),loss = loss, metrics=['accuracy'])


In [7]:
import datetime
import time
log_dir = "logs/fit_binary/" + 'mobilenetv2_ver_1.0'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [8]:
history = model.fit(train_images,
            steps_per_epoch = train_images.n//train_images.batch_size,
            epochs = 20,
            validation_data = val_images,
            validation_steps = val_images.n//val_images.batch_size,
            verbose = 1,
            callbacks=[tensorboard_callback])


Epoch 1/20
 1/28 [>.............................] - ETA: 0s - loss: 0.9974 - accuracy: 0.6000WARNING:tensorflow:From C:\Users\Janmejai\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
28/28 [==============================] - 29s 1s/step - loss: 1.8690 - accuracy: 0.7525 - val_loss: 2.9323 - val_accuracy: 0.7697
Epoch 2/20
28/28 [==============================] - 21s 764ms/step - loss: 1.8403 - accuracy: 0.8093 - val_loss: 1.2871 - val_accuracy: 0.8545
Epoch 3/20
28/28 [==============================] - 17s 593ms/step - loss: 1.1745 - accuracy: 0.8603 - val_loss: 1.7595 - val_accuracy: 0.7636
Epoch 4/20
28/28 [==============================] - 16s 583ms/step - loss: 1.5512 - accuracy: 0.8282 - val_loss: 1.0079 - val_accuracy: 0.8727
Epoch 5/20
28/28 [==============================] - 23s 819ms/st

In [9]:
import tensorflow
initial_lr = 1e-4
loss = "categorical_crossentropy"
model.compile(tensorflow.keras.optimizers.Adam(lr=initial_lr),loss = loss, metrics=['accuracy'])
history = model.fit(train_images,
            steps_per_epoch = train_images.n//train_images.batch_size,
            epochs = 10,
            validation_data = val_images,
            validation_steps = val_images.n//val_images.batch_size,
            verbose = 1,
            callbacks=[tensorboard_callback])

Epoch 1/10
28/28 [==============================] - 22s 773ms/step - loss: 1.1069 - accuracy: 0.8996 - val_loss: 1.6199 - val_accuracy: 0.8788
Epoch 2/10
28/28 [==============================] - 21s 756ms/step - loss: 0.9546 - accuracy: 0.9170 - val_loss: 1.6035 - val_accuracy: 0.8727
Epoch 3/10
28/28 [==============================] - 21s 737ms/step - loss: 0.9726 - accuracy: 0.9039 - val_loss: 1.3201 - val_accuracy: 0.8909
Epoch 4/10
28/28 [==============================] - 17s 620ms/step - loss: 1.0159 - accuracy: 0.8952 - val_loss: 1.4202 - val_accuracy: 0.8909
Epoch 5/10
28/28 [==============================] - 17s 610ms/step - loss: 0.8817 - accuracy: 0.9098 - val_loss: 1.3711 - val_accuracy: 0.8788
Epoch 6/10
28/28 [==============================] - 17s 593ms/step - loss: 0.9826 - accuracy: 0.9039 - val_loss: 1.3631 - val_accuracy: 0.9030
Epoch 7/10
28/28 [==============================] - 17s 602ms/step - loss: 0.7562 - accuracy: 0.9214 - val_loss: 1.2222 - val_accuracy: 0.8848

In [10]:
model.save('mobilenetv2.h5')

In [14]:
# model.save_weights('mobilenetv2_ver_1.0')

In [11]:
# !pip install gradio

# Gradio is an amazing tool to check your model.
# Running this cell will start a local server.
# You can then go the link and check on your images.
import gradio as gr
labels = ['COVID', 'Normal']

def classify_image(inp):
    inp = inp.reshape((-1, 244, 244, 3))
    inp = tensorflow.keras.applications.mobilenet_v2.preprocess_input(inp)
    prediction = model.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(2)}



image = gr.inputs.Image(shape=(244, 244, 3))
label = gr.outputs.Label(num_top_classes=2)

gr.Interface(fn=classify_image, inputs=image, outputs=label, capture_session=True).launch()

Running locally at: http://127.0.0.1:7860/


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x24423ecf308>,
 'http://127.0.0.1:7860/',
 None)

In [12]:
model.save('saved_model/my_model') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [13]:
saved_model_dir = 'saved_model/my_model'

In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)


In [15]:
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

9388636